# Guide to obtain a two-body PEF
## 1. Define and set variables

<p style="color:red;">Note: This notebook has been updated as of 05/07/2019 to work with the new postgres database. In order to help users migrate their notebooks to the new format, we have made notes in RED where this notebook has been changed.</p>

### 1.1. Import the python library
Remember that in order to import the library without any errors, you need to perform the following operations in the bash terminal from which you are running the notebook. If you didn't do it, please, close the notebook and write in a bash terminal:
```sh
cd HOME/DIRECTORY/OF/POTENTIAL_FITTING
source install.sh
```
Now the following command should run without any problem.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import potential_fitting

### 1.2. Define variables and filepaths

The working directory is the path where the files should be. If they are not in the working directory, the full path to the file should be provided.

#### Files that will be read by the code

Settings files contain the information regarding the level of theory at which calculations should be performed, and information about the symmetry. The settings files has five different sections:

- First section is named `[files]`, and has only the `log_path` variable, which specifies the folder that will be used to dump all the logfiles, outputs, and so on.

<p style="color:red;">Note: method, basis, and cp are no longer specified in the `.ini` file for config_generator. These parameters will now be passed directly into the geometry optimization and normal modes generation function.</p>
- Second section corresponds to the settings for the generation of the configurations that will be used in the training and test sets. It is names as `[config_generator]`, and it contains several variables that need to be set:
  * `code = ####` specifies the software that will be used to perform the optimization and the normal mode calculation. The optimized geometry and the frequencies will be used to generate configurations for the training set. **Available options are: **
    + `code = qchem` [See manual here](http://www.q-chem.com/qchem-website/manual/qchem50_manual/index.html)
    + `code = psi4` [See manual here](http://www.psicode.org/psi4manual/1.2/index.html)
  * `geometric = True/False` and `linear = True/False` will define which kind of progression for the temperature and the normal mode coordinates we use. **One, and only one, must be True.** **NOTE: WHAT IS GEOMETRIC AND WHAT IS LINEAR?**
  

<p style="color:red;">Note: method, basis, and cp are no longer specified in the `.ini` file for energy_calculation. These parameters will now be passed directly into the init_database function. Additionally the SMILES property is new.</p>
- Third section corresponds to the energy calculation settings, labeled as `[energy_calculator]`. The code must be specified again, and can be different than in the `[config_generator]` section.
  + `code = ####` has been previously explained.

- The two next sections correspond to the computational details of the QC package. As for today, the sections are `[qchem]` and `[psi4]`, with the following keywords:
  + `memory = ####` will set the memory (RAM memory) to reserve for the calculation. For DFT, 2 or 4 GB (`memory = 4GB`) should be enough. If we want CCSD(T) calculations, we might have to increase it to 8 or 16 GB.
  + `num_threads = ####` will set the number of cores to use in the calculation.
- Memory and num_threads in the `[psi4]` section go for psi4, and the same of qchem.

- The last section contains information about the system we are fitting such as the name of the monomers, number of atoms...
  + `names = ####` will define the **name** we want to give to the monomers. If we have N monomers, we need N names separated by commas. The name can be anything, but it should be enough to identify which monomer is which. For example, `names = A,B` is a valid entry for a dimer, but a poor choice for remembering which monomer is A and which is B. If we have a amonium cation -- water dimer, a good choice would be `names = NH4+, H2O`.
  + `fragments = ####` will define the **number of atoms** that each monomer has. It must be a positive integer, and if there is more than one monomer, should be separated by commas. **The order here must be the same as the order in `names`**.
  + `charges = ####` will define the charge of each monomer. If there is more than one monomer, should be separated by commas. **The order here must be the same as the order in `names`**.
  + `spins = ####` will define the spin multiplicity (`S = 0.5 * s + 1)`, where `s` is the number of unpaired electrons) of each monomer. If there is more than one monomer, should be separated by commas. **The order here must be the same as the order in `names`**.
  + `tag = ####` specifice a tag that will be added to the configurations in the database. **Set it at `tag = none` if no tag is needed**.
  + `symmetry = ####` requires the atom identity for the monomers. Some examples are `symmetry = A1B4` for methane monomer, `symmetry = A1B2,A1B2` for a CO2 dimer, `symmetry = A6B6,C1D2` for a benzene -- water dimer without lone pairs, and `symmetry = A1B2Z2,C1D2` for a H2O -- SO2 dimer with lone pairs. The rules are the following:
    * Symmetry names must be written in capital letters and start with A for the first atom of the first monomer. Any new atom type will be assigned the next letter of the alphabet.
    * Exchangable atoms must have the same label, even if they are in different molecules.
    * As for today, no more than 9 atoms of the same atom type is accepted.
    * If there are virtual sites such as lone pairs that will play a role in the polynomials, they must be labels with letters X, Y, or Z.
  + `SMILES = ###` The smile strings for each monomer, comma delimited. Some examples are `SMILE = C(H)(H)(H)H` for methane monomer, `SMILES = C(O)O,C(O)O` for a CO2 dimer, `SMILES = C12C3C4C5C6C17.H2.H3.H4.H5.H6.H7,O(H)H` for a benze - water dimer.
    * More info about SMILE strings and how to write them can be found here: https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system.
    * For now, all Hs must be stated explicity.
    * For now, ignore all charges when writing your SMILE string.
  + `use_mbpol = ####` specifies for which monomers we will use the partridge-schwenke surface for the charges and the energies. It is a list of integers, separated by a comma. `0` means we do not use mb-pol, `1` means we use mb-pol. As an example: `use_mbpol = 0,1` will use the charges and polarizabilities obtained from QChem for monomer 1, but use MB-pol charges and pols for monomer 2.

In [ ]:
# Settings for monomer 1
mon1_settings = "monomer1_settings.ini"

# Settings for monomer 2
mon2_settings = "monomer2_settings.ini"

# Settings for the dimer
dimer_settings = "dimer_settings.ini"

method = "HF"
basis = "STO-3G"
cp = False

Unoptimized geometries of the two monomers are inputed as an [XYZ formatted file](https://en.wikipedia.org/wiki/XYZ_file_format). 

In [ ]:
# XYZ file that contains the unoptimized geommetry of monomer 1
unopt_mon1 = "monomer1.xyz"

# XYZ file that contains the unoptimized geommetry of monomer 2
unopt_mon2 = "monomer2.xyz"

#### Files that will be written by the code

<p style="color:red;">Note: the optimized .xyz file no longer must end with `.opt.xyz`. However it is not a problem if it does.</p>
Optimized geommetries of the two monomers are outputed as an [XYZ formatted file](https://en.wikipedia.org/wiki/XYZ_file_format).

In [ ]:
# XYZ file that contains the optimized geommetry of monomer 1
opt_mon1 = unopt_mon1.split(".xyz")[0] + ".opt.xyz"

# XYZ file that contains the optimized geommetry of monomer 2
opt_mon2 = unopt_mon2.split(".xyz")[0] + ".opt.xyz"

# File where normal modes of monomer 1 will be outputed
normal_modes_mon1 = "monomer1_normal_modes.dat"

# File where normal modes of monomer 2 will be outputed
normal_modes_mon2 = "monomer2_normal_modes.dat"

# Dimer configuration to optimize
unopt_dim = "dimer.xyz"

# Optimized configuration of the dimer
opt_dim = unopt_dim.split(".xyz")[0] + ".opt.xyz"

# File where normal modes of dimer will be outputed
normal_modes_dim = "dimer_normal_modes.dat"

Training and test set files. The `training configs` and `test_configs` files will contain the configurations generated by the training set generation functions. Only the geometry (i.e., the coordinates of all atoms for each configuration) will be stored in those files. Later on, we will calculate the energies for each of these configurations, and create the files in the format that will be inputed to the fitting code. These new files are going to be defined in `training_set` and `test_set`. The coordinates in these files will be the same as in the initial `training configs` and `test_configs`, but now the comment line will be filled with the energies needed by the fitting code.

In [ ]:
# XYZ file with the configurations of the training set
rigid_training_configs = "training_configs_rigid.xyz"
flexible_training_configs = "training_configs_flexible.xyz"
normal_modes_training_configs = "training_configs_normal_modes.xyz"

# XYZ file with the configurations of the test set
rigid_test_configs = "test_configs_rigid.xyz"
flexible_test_configs = "test_configs_flexible.xyz"
normal_modes_test_configs = "test_configs_normal_modes.xyz"

# XYZ file with the training set that the codes need to perform the fit
# Configurations are the same as training_configs but this file
# has the energies in the comment line
training_set = "training_set.xyz"

# XYZ file with the test set that the codes need to perform the fit
# Configurations are the same as test_configs but this file
# has the energies in the comment line 
test_set = "test_set.xyz"

# XYZ file with the configurations of distorted monomers 1 and 2
mon1_distorted_initial = "monomer1_distorted_configs_initial.xyz"
mon2_distorted_initial = "monomer2_distorted_configs_initial.xyz"

# Screened XYZ files with energy lower than the threshold
mon1_distorted_screened = "monomer1_distorted_configs_screened.xyz"
mon2_distorted_screened = "monomer2_distorted_configs_screened.xyz"

The information about the training, test, energies... is stored in an `postgreSQL` database. In principle there is no need to interact with this database, since everything is automatized, but you might want to retrieve some information at some point.
<p style="color:red;">Note: the parameter `database` has been removed, as the PostgreSQL database is not saved locally. Instead, it is hosted remotely. You now must include a database_config `.ini` file with the information needed to login to the database. Also, the client_name parameter is new.</p>

The database_config `.ini` file should contain one section `[database]` with 5 properties:
* `host`: The address of the server where the database is hosted.
* `port`: The port used to connect to the database.
* `database`: The name of the database.
* `username`: Your username to connect to the database.
* `password`: Your password to connect to the database.

For now use these parameters:

* `host`: piggy.pl.ucsd.edu
* `port`: 5432
* `database`: potential_fitting
* `username`: potential_fitting
* `password`: Please contact Ethan or Kaushik for the password.

The username potential_fitting was established as a general username that anyone who only needs basic access to the database can use. Alternatively, each user has their own username and password you can use. For most of you, this should be the same as your ucsd email prefix and password.

The file database.ini does not exist in the git repo, so you will have to create a file and update the variable below to be its filepath. For some reason, python doesn't like it when you use `~` to specify your home directory, so provide a relateive or absolute path instead. It is recommended that you create the file in your home directory.

<h3 style="color:red;">Make sure only you have read access to this file using the chmod command or else anyone on our fileserver will be able to see your password and <b>PLEASE DO NOT ACCIDENTALLY COMMIT A FILE CONTAINING YOUR PASSWORD VIA GIT!</b></h3>

cleint_name is used in the database to track what machines performed what calculations. Please use something that indicates where you are running the calculations.

In [ ]:
# PostgreSQL database that stores structures and energies
database_config = "database.ini"
client_name = "motzu the survivor"

These files will be used by the polynomial generation functions. 
- `poly_in` is the name of the file that will contain all the information about the polynomials: distances, variables, filters... Later on, once it is created, extra filters can be added. How to add them will be explained later in the tutorial.
- `molecule_in` is the symmetry name of your system. It must match the symmetry specified in the settings file, and must follow the same rules. If you are fitting a system larger than a monomer (two-body, three-body...), this is the monomer names separated by an underscore (`_`). As an example, for a CO2 dimer, `molecule_in = A1B2_A1B2`, while for a NH4+ -- H2O dimer, `molecule_in = A1B4_C1D2`.
- `poly_directory` is the folder that will be created and will contain all the polynomial files generated.
- `config` is the name that the file containing all the chemical and physical information about monomers such as charges, C6 coefficients, polarizabilities... It will be generated by the code.
- `polynomial_order` specifies the maximum order of the polynomials. 

In [ ]:
# Input file for the polynomial generation
poly_in = "poly.in"

# Monomers 1 and 2 separated by '_'
molecule_in = "A4B1_C2D1X2"

# Directory where the polynomials will be generated
poly_directory = "polynomial_generation"

# Configuration file that contains all the monomer 
# and dimer information. Will be used to generate the 2B codes.
config = "config.ini"

# Degree of the polynomials
polynomial_order = 2

#### Directories for the different sections

These variables specify the directories where the fitting code for each type of PEF is going to be created.
- `ttmnrg_directory` will contain the code that fits TTM-nrg PEFs for the system specified.
- `mbnrg_directory` will contain the code that fits MB-nrg PEFs for the system specified.
- `results_prefix` is the prefix that the final output files for the mb-nrg fit will have.

In [ ]:
# Directory where ttm-nrg fitting code will be stored
ttmnrg_directory = "ttm-nrg_fit"

# Directory where mb-nrg fitting code will be stored
mbnrg_directory = "mb-nrg_fit"

# Prefix for mb-nrg output files
results_prefix = "mb-nrg"

#### Multiple variables that will be used later

<p style="color:red;">Note: a new variable is here, molecule_name. It is used in generate_2b_training_set() For now, the molecule name should always be the names of the fragments delimerated by "-"s in alphabetical order.</p>

In [ ]:
# Number of configurations in the 2b training_set
num_rigid_training_configs = 200
num_flexible_training_configs = 200
num_normal_modes_training_configs = 100

# Number of configurations in the 2b test set
num_rigid_test_configs = 100
num_flexible_test_configs = 100
num_normal_modes_test_configs = 50

# Number of distorted configurations to generate for each monomer
num_mon1_distorted = 50
num_mon2_distorted = 50

# Maximum energy allowed for distorted monomers (in kcal/mol)
mon_emax = 30.0

# Maximum binding energy allowed
bind_emax = 500.0

# Seeds to be used in the configuration generation to ensure different
# configurations for training and test
seed_training = 12345
seed_test = 54321

# Seeds for the monomer distortion configuration
seed_mon1 = 666
seed_mon2 = 69

# Minimum acceptable distance between the center of mass of
# the two monomers in the two-body training set
min_d_2b = 2.0

# Maximum acceptable distance between the center of mass of
# the two monomers in the two-body training set
max_d_2b = 10.0

# Minimum distance acceptable between two atoms of different monomers
min_inter_d = 1.2

# Name of the entire molecule
molecule_name = "H2O-NH4+"

# Names of monomer 1 and 2
mon1_name = "NH4+"
mon2_name = "H2O"

# IDs of the monomers (should be consistent with the 1B id for each)
mon1_id = "nh4+"
mon2_id = "h2o"

# Number of TTM-nrg fits to perform
num_ttm_fits = 1

# Number of TTM-nrg fits to perform
num_mb_fits = 10

## 2. Geometry optimization and normal mode calculation

### 2.1. Monomers

Performs a geometry optimization of both monomers at the level of theory specified in `monomer1_settings.ini` and `monomer2_settings.ini`. **Before running these commands** please make sure that the specifications in the sections `[config_generator]` and `[molecule]` of the corresponding `settings.ini` file are correct and consistent.

In [ ]:
# Optimize monomer 1
potential_fitting.optimize_geometry(mon1_settings, unopt_mon1, opt_mon1, method, basis)

# Get its normal modes
potential_fitting.generate_normal_modes(mon1_settings,opt_mon1,normal_modes_mon1, method, basis)

In [ ]:
# Optimize monomer 2
potential_fitting.optimize_geometry(mon2_settings, unopt_mon2, opt_mon2, method, basis)

# Get its normal modes
potential_fitting.generate_normal_modes(mon2_settings,opt_mon2,normal_modes_mon2, method, basis)

### 2.2. Dimer

Performs a geometry optimization of the dimer at the level of theory specified in `dimer_settings.ini`. **Before running these commands** please make sure that the specifications in the sections `[config_generator]` and `[molecule]` of the corresponding `settings.ini` file are correct and consistent. First we have to make a file that contains the unoptimized dimer.

In [ ]:
potential_fitting.generate_2b_configurations(dimer_settings, opt_mon1, opt_mon2,1,unopt_dim,2.0,3.1,1.0)

Now we perform the optimization and normal mode calculation.

In [ ]:
# Optimize dimer
potential_fitting.optimize_geometry(dimer_settings, unopt_dim, opt_dim, method, basis)

# Get its normal modes
potential_fitting.generate_normal_modes(dimer_settings,opt_dim,normal_modes_dim, method, basis)

## 3. Training and test set generation

### 3.1. Rigid training and test sets

#### 3.1.1. Generate configurations 

Generates rigid 2b configurations using the optimized geommetry previously calculated. The configrations that we have generated will be stored in an XYZ formatted file with the names we have previously given.

In [ ]:
# Training Set
potential_fitting.generate_2b_configurations(dimer_settings, opt_mon1, opt_mon2, num_rigid_training_configs, rigid_training_configs, min_d_2b, max_d_2b, min_inter_d, seed_training)

In [ ]:
# Test Set
potential_fitting.generate_2b_configurations(dimer_settings, opt_mon1, opt_mon2, num_rigid_test_configs, rigid_test_configs, min_d_2b, max_d_2b, min_inter_d, seed_test)

#### 3.1.2. Add configurations to the database

The configurations generated in the previous step will be added to the database. **This step will only add the configurations, not calculate the energy**.

<p style="color:red;">Note: the arguments for the init_database function have changed. The method, basis, and cp are now specified here instead of in mon_settings. Additionally, the database filepath argument has been exchanged for the database_config. Also, instead of specifying tags as a keyword argument, it is specified after cp, multiple tags may be included. The argument optimized has been added. If true, all geometries in the config file will be considered optimized geometries.</p>

The method, basis, and cp need not be the same as used for the geometry optimization. The optimization and normal modes calculation can be performed at a different level of theory than the energy evaluation. **The recommendation is to use the same settings**, but is up to the user to do it so.

`cp = True/False` will define if, in case it is applicable (two-body and three-body calculations), the counter-poise correction will be used or not. When the basis set is relatively small (6-31g, cc-pvdz, ...), there might be an artificial interaction between the molecules. Sometimes, using basis functions from the one monomer to describe the electrons of the other monomer stabilizes the energy, yielding a false larger interaction energy. It is possible to remove this effect, but the calculation will acquire a higher cost.

In [ ]:
# Add dimer training set configurations
potential_fitting.init_database(dimer_settings, database_config, rigid_training_configs, method, basis, cp, "training", optimized = False)
# Add monomer 1 optimized geommetry to database (needed for binding energy)
potential_fitting.init_database(mon1_settings, database_config, opt_mon1, method, basis, cp, "training", optimized = True)

# Add monomer 2 optimized geommetry to database (needed for binding energy)
potential_fitting.init_database(mon2_settings, database_config, opt_mon2, method, basis, cp, "training", optimized = True)

In [ ]:
# Add dimer test set configurations
potential_fitting.init_database(dimer_settings, database_config, rigid_test_configs, method, basis, cp, "test", optimized = False)

# Add monomer 1 optimized geommetry to database (needed for binding energy)
potential_fitting.init_database(mon1_settings, database_config, opt_mon1, method, basis, cp, "test", optimized = True)

# Add monomer 2 optimized geommetry to database (needed for binding energy)
potential_fitting.init_database(mon2_settings, database_config, opt_mon2, method, basis, cp, "test", optimized = True)

### 3.2. Normal modes training and test sets

#### 3.2.1. Configurations from dimer normal modes

Obtain configurations by using the normal mode coordinate of the minimum energy of the dimer.

In [ ]:
# Get some for the training set
potential_fitting.generate_normal_mode_configurations(dimer_settings, opt_dim, normal_modes_dim, normal_modes_training_configs, num_normal_modes_training_configs, seed_training)

In [ ]:
# Get some for the test set
potential_fitting.generate_normal_mode_configurations(dimer_settings, opt_dim, normal_modes_dim, normal_modes_test_configs, num_normal_modes_test_configs, seed_test)

#### 3.2.2. Add configurations to the database

In [ ]:
# Add dimer training set configurations
potential_fitting.init_database(dimer_settings, database, normal_modes_training_configs, tag = "training")

In [ ]:
# Add dimer test set configurations
potential_fitting.init_database(dimer_settings, database, normal_modes_test_configs, tag = "test")

### 3.3. Flexible training and test sets

#### 3.3.1. Obtain distorted monomer configurations

In [ ]:
# Monomer 1 configurations
potential_fitting.generate_normal_mode_configurations(mon1_settings, opt_mon1, normal_modes_mon1, mon1_distorted_initial, num_mon1_distorted, seed_mon1)

In [ ]:
# Monomer 2 configurations
potential_fitting.generate_normal_mode_configurations(mon2_settings, opt_mon2, normal_modes_mon2, mon2_distorted_initial, num_mon2_distorted, seed_mon2)

#### 3.3.2. Add them to the database

In [ ]:
# Monomer 1 configurations
potential_fitting.init_database(mon1_settings, database_config, mon1_distorted_initial, method, basis, cp, "mon1", optimized = False)

# Monomer 1 optimized geommetry
potential_fitting.init_database(mon1_settings, database_config, opt_mon1, method, basis, cp, "mon1", optimized = True)

# Monomer 2 configurations
potential_fitting.init_database(mon2_settings, database_config, mon2_distorted_initial, method, basis, cp, "mon2", optimized = False)

# Monomer 2 optimized geommetry
potential_fitting.init_database(mon2_settings, database_config, opt_mon2, method, basis, cp, "mon2", optimized = True)

#### 3.3.3. Get the energy
<p style="color:red;">Note: the arguments for the fill_database function have changed. The database filepath argument has been exchanged for the database_config. Additionally, the client_name must now be specified. If desired, there is an optional argument calculation_count, when set to an integer, it limits the number of calculations to perform.</p>

In [ ]:
# Fill database for mon1
potential_fitting.fill_database(mon1_settings, database_config, client_name, "mon1", "mon2", calculation_count = None)

#### 3.3.4. Generate configurations with monomer energy lower than threshold

<p style="color:red;">Note: the arguments for the generate_1b_training_set function have changed. The database filepath argument has been exchanged for the database_config. Additionally, method, basis, and cp must now be specified. Also, instead of specifying tags as a keyword argument, it is specified after cp, multiple tags may be included.</p>

In [ ]:
# Get mon1 screened configurations
potential_fitting.generate_1b_training_set(mon1_settings, database_config, mon1_distorted_screened, mon1_name, method, basis, cp, "mon1", e_min = 0, e_max = mon_emax)

# Get mon2 screened configurations
potential_fitting.generate_1b_training_set(mon2_settings, database_config, mon2_distorted_screened, mon2_name, method, basis, cp, "mon2", e_min = 0, e_max = mon_emax)

#### 3.3.5. Generate distorted training set

In [ ]:
# Training Set
potential_fitting.generate_2b_configurations(dimer_settings, mon1_distorted_screened, mon2_distorted_screened, num_flexible_training_configs, flexible_training_configs, min_d_2b, max_d_2b, min_inter_d, seed_training + 1)

In [ ]:
# Training Set
potential_fitting.generate_2b_configurations(dimer_settings, mon1_distorted_screened, mon2_distorted_screened, num_flexible_test_configs, flexible_test_configs, min_d_2b, max_d_2b, min_inter_d, seed_test + 1)

#### 3.3.6. Add configurations to the database

<p style="color:red;">Note: the arguments for the init_database function have changed. The method, basis, and cp are now specified here instead of in mon_settings. Additionally, the database filepath argument has been exchanged for the database_config. Also, instead of specifying tags as a keyword argument, it is specified after cp, multiple tags may be included. The argument optimized has been added. If true, all geometries in the config file will be considered optimized geometries.</p>

In [ ]:
# Add dimer training set configurations
potential_fitting.init_database(dimer_settings, database_config, flexible_training_configs, method, basis, cp, "training", optimized = False)

In [ ]:
# Add dimer test set configurations
potential_fitting.init_database(dimer_settings, database_config, flexible_test_configs, method, basis, cp, "test", optimized = False)

### 3.4. Calculate energy

Loops through every uncalculated energy in the database and calculates it. This will take a while depending on what method/basis you use.
<p style="color:red;">Note: the arguments for the fill_database function have changed. The database filepath argument has been exchanged for the database_config. Additionally, the client_name must now be specified. If desired, there is an optional argument calculation_count, when set to an integer, it limits the number of calculations to perform.</p>

In [ ]:
potential_fitting.fill_database(dimer_settings, database_config, client_name, "training", "test", calculation_count = None)

### 3.5. Training set and Test set generation

Generates the training set file in the format that will be needed in the fitting codes. If your database contains energies computed with a variety of methods/basis, **only one method and basis can be used in the same training set**. The format of the training set is the same as the configurations generated for the training set in previous steps. The difference is that now, the comment line will have the binding, interaction, and one body energies of that configuration.

<p style="color:red;">Note: the arguments for the generate_2b_training_set function have changed. The database filepath argument has been exchanged for the database_config. The molecule name must also now be specified in addition to the two monomer names. Additionally, method, basis, and cp must now be specified. Also, instead of specifying tags as a keyword argument, it is specified after cp, multiple tags may be included.</p>

In [ ]:
# Generate training set
potential_fitting.generate_2b_training_set(dimer_settings, database_config, training_set, molecule_name, method, basis, cp, "training", e_bind_max = bind_emax, e_mon_max = mon_emax)

# Generate test set
potential_fitting.generate_2b_training_set(dimer_settings, database_config, test_set, molecule_name, method, basis, cp, "test", e_bind_max = bind_emax, e_mon_max = mon_emax)

## 4. Generate polynomial files

### 4.1. Generate polynomial input file

This call generates a polynomial input file based on the symmetry of the dimer specified. 

*Note. Write some more info and doc for the input. Filters by default, new filters that can be added...*
<p style="color:red;">Note: the database filepath argument has been exchanged for the database_config in generate_poly_input_from_database </p>

In [ ]:
potential_fitting.generate_poly_input(dimer_settings, molecule_in, poly_in)

### 4.2. Generate maple input files

Generate polynomials of the degree specified at the beginning, based on the polynomial input file that we have generated in the previous step.

In [ ]:
potential_fitting.generate_polynomials(dimer_settings, poly_in, polynomial_order, poly_directory)

### 4.3. Optimize the polynomial evaluation

The maple input files define the non optimized polynomials. The polynomials can sometimes be large, and **Maple** is able to optimize them to perform the minimum number of floating point operations (FLOPs). It will output three different files. One with non-optimized polynomials, one with optimized polynomials with gradient evaluation, and one without gradient evaluation.

In [ ]:
potential_fitting.execute_maple(dimer_settings, poly_directory)

## 5. Obtain monomer and dimer properties

In order to perform the fit, charges, polarizabilities, C6, and other properties of the dimer have to be calculated. This is done, for now, with the software **QChem**. This instruction will compute these properties for you. The predefined basis set and method is wb97m-v/avtz. This step can take a long time if the molecule is large.

After the calculation is completed, all the information needed for the fits (both MB-nrg and TTM-nrg) will be added in the configuration file specified at the beggining.

In [ ]:
potential_fitting.generate_fit_config(dimer_settings, molecule_in, config, opt_mon1, opt_mon2)

## 6. TTM-nrg fit

### 6.1. Obtain and compile the fitting code

Generates a two-body TTM-nrg fitting code for this specific dimer. It will use all the specifications defined in the config file, and it will create a C++ software that will be able to perform the fit.

In [ ]:
potential_fitting.generate_2b_ttm_fit_code(dimer_settings, config, molecule_in, ttmnrg_directory)

Compile the fit code in fit_directory

In [ ]:
potential_fitting.compile_fit_code(dimer_settings, ttmnrg_directory)

### 6.2 Perform the fit

This step will perform multiple fits **(NOTE: allow the user to specify)** and it will keep the fit with best RMSD. After that, the config file will be updated with the TTM-nrg parameters that we have obtained, so we can proceed to MB-nrg fit.

In [ ]:
potential_fitting.fit_2b_ttm_training_set(dimer_settings, ttmnrg_directory + "/fit-2b-ttm", training_set, ttmnrg_directory, config, num_ttm_fits)

## 7. MB-nrg fit

### 7.1. Obtain and compile the fitting code

Using the TTM-nrg parameters obtained previously, we will generate a similar code for the two-body MB-nrg fit.

In [ ]:
potential_fitting.generate_2b_fit_code(dimer_settings, config, poly_in, poly_directory, polynomial_order, mbnrg_directory)

And we compile it.

In [ ]:
potential_fitting.compile_fit_code(dimer_settings, mbnrg_directory)

### 7.2. Perform the fit

We perform **(multiple?)** fits, keep the best one, and keep those results.

In [ ]:
potential_fitting.fit_2b_training_set(dimer_settings, mbnrg_directory + "/fit-2b", training_set, mbnrg_directory, results_prefix + ".nc", num_mb_fits)

## 8. Visualize the results

Finally, we can plot the correlation plots for the training and test sets, along with the error, using this helper function.

In [ ]:
%matplotlib inline
potential_fitting.fitting.fit_visualizer.make_2b_graphs(dimer_settings, ttmnrg_directory + "/eval-2b-ttm", ttmnrg_directory + "/ttm-params.txt", mbnrg_directory + "/eval-2b", results_prefix + ".nc", database_config, molecule_name, method, basis, cp, "test")

## 9. Generate files for Clusters Ultimate

Finally, generate the software interface to use this PEF in Clusters_Ultimate

In [ ]:
potential_fitting.fitting.generate_software_files_2b(dimer_settings, molecule_in, poly_directory, polynomial_order, mbnrg_directory, config, "fit-2b.cdl", mon1_id, mon2_id)